In [ ]:
import os


# Get the user's home directory Documents folder
path_dir = os.path.join(os.path.expanduser('~'), 'Documents/')
path_data_dump = path_dir + "VSCode_dump/"

print(f'path_dir: {path_dir}')
print(f'path_data_dump: {path_data_dump}')

In [1]:
import pandas as pd
from myUtils import pickle_load, pickle_dump, delete_all_rows
from yf_utils import split_train_val_test, random_slices, lookback_slices
from yf_utils import top_set_sym_freq_cnt, get_grp_top_syms_n_freq
from yf_utils import eval_grp_top_syms_n_freq


pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_colwidth", 20)
pd.set_option("display.width", 500)  # code-runner format

# path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
# path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"

### Run below cell for Colab

In [2]:
#######################################################################
## SELECT RUN PARAMETERS.async Parameters can also be passed using papermill by running yf_7_freq_cnt_pm_.ipynb
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# select run type
run_type = "train"
# run_type = 'validate'
# run_type = "test"

# number of max lookback tuples to create for iloc iloc_start_train:iloc_end_train:iloc_end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms
# n_samples = 400
n_samples = 2

# for training, the number of days to lookback from iloc max-lookback iloc_end_train
# days_lookbacks = [15, 30, 60]
# days_lookbacks = [30, 60, 120]
days_lookbacks = [15, 30, 60, 120]
days_lookbacks.sort()

# number of days in dataframe to evaluate effectiveness of the training, days_eval = len(df_eval)
#  days_eval = 4 means buy at close on 1st day after the signal, hold for 2nd and 3rd day, sell at close on 4th day
days_eval = 4
# days_eval = 5

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20
# n_top_syms = 5

# slice starts and ends for selecting the best performing symbols
syms_start = 0
syms_end = 10


##########################################################
# fp_df_eval_results = f"df_eval_results_{run_type}"
fp_df_eval_results = "test_df_eval"
##########################################################

#### Delete all rows in df_eval_results ##################
delete_all_rows(path_data_dump, fp_df_eval_results)
##########################################################

In [3]:
# Parameters
verbose = True
store_results = False
n_top_syms = 20
syms_start = 0
syms_end = 10
run_type = "test"
fp_df_eval_results = "df_eval_results_test"
n_samples = 2
days_eval = 4
days_lookbacks = [15, 30, 60, 120]


In [4]:
path_data_dump

'C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/'

In [5]:
print(f"verbose : {verbose }")
print(f"store_results: {store_results}")
print(f"run_type: {run_type}")
print(f"n_samples: {n_samples}")
print(f"days_lookbacks: {days_lookbacks}")
print(f"days_eval: {days_eval}")
print(f"n_top_syms: {n_top_syms}")
print(f"syms_start: {syms_start}")
print(f"syms_end: {syms_end}")
print(f"fp_df_eval_results: {fp_df_eval_results}")

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = split_train_val_test(df_close_clean)

verbose : True
store_results: False
run_type: test
n_samples: 2
days_lookbacks: [15, 30, 60, 120]
days_eval: 4
n_top_syms: 20
syms_start: 0
syms_end: 10
fp_df_eval_results: df_eval_results_test


In [6]:
# max_days_lookbacks = max(days_lookbacks)
# print(f"max_days_lookbacks: {max_days_lookbacks}")

# Load df according to run_type
if run_type == "train":
    df = df_train.copy()
elif run_type == "validate":
    df = df_val.copy()
elif run_type == "test":
    df = df_test.copy()
else:
    msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop)

# Print dataframe for the run, and lengths of other dataframes
print(f"run_type: {run_type}, df.tail(3):\n{df.tail(3)}\n")
len_df = len(df)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f"run_type: {run_type}, len(df): {len(df)}")
print(
    f"len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test}"
)

# return n_samples slices
max_lookback_slices = random_slices(
    len_df,
    n_samples=n_samples,
    days_lookback=max(days_lookbacks),
    days_eval=days_eval,
    verbose=False,
)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = lookback_slices(
    max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False
)

run_type: test, df.tail(3):
                     A         AA    AAL        AAP        AAPL         AB        ABBV    ABR         ABT       ACGL   ACHC       ACIW        ACLS  ACRS        ADBE  ...    XRX         XYL       YELP         YUM       YUMC         YY         ZBH        ZBRA         ZD     ZG       ZION        ZTO         ZTS       ZUMZ    ZWS
Date                                                                                                                                                                  ...                                                                                                                                                              
2024-01-03  131.160004  31.490000  12.95  61.000000  184.250000  30.370001  160.460007  14.69  109.519997  76.940002  76.68  29.530001  120.919998  1.12  571.789978  ...  15.84  111.559998  46.009998  129.100006  42.950001  32.970001  120.470001  252.520004  65.860001  53.98  42.000000  21.059999  192.929993  18.93

### Run below cell to use test data to debug code, results should match yf_test.ipynb

In [7]:
# ####### RUN THIS CELL TO USE TEST DATA TO DEBUG CODE, RESULTS SHOULD MATCH yf_test.ipynb ################# 

# # load test data to debug code if needed
# test_dic = pickle_load(path_data_dump, 'yf_test_dic')
# for i, key in enumerate(test_dic.keys()):
#   print(f'{i}: {key}')

# # convert dictionary entries into variables
# locals().update(test_dic)
# del df_eval_results  # delete df_eval_results loaded from locals().update(test_dic)

# # Delete all rows in df_eval_results
# delete_all_rows(path_data_dump, fp_df_eval_results)

In [8]:
max_days_lookbacks = max(days_lookbacks)
print(f"max_days_lookbacks: {max_days_lookbacks}")

if verbose:
    print(f"number of max_lookback_slices is equal to n_samples = {n_samples}")
    print(f"max_lookback_slices:\n{max_lookback_slices}\n")
    print(f"number of sets in sets_lookback_slices is equal to n_samples = {n_samples}")
    print(f"sets_lookback_slices:\n{sets_lookback_slices}\n")
    print(f"days_lookbacks: {days_lookbacks}")
    print(
        f'number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): {len(days_lookbacks)}'
    )

# #### Generate grp_top_set_syms_n_freq. It is a list of sub-lists, e.g.:
#  - [[('AGY', 7), ('PCG', 7), ('KDN', 6), ..., ('CYT', 3)], ..., [('FCN', 9), ('HIG', 9), ('SJR', 8), ..., ('BFH', 2)]]
# #### grp_top_set_syms_n_freq has n_samples sub-lists. Each sub-list corresponds to a tuple in the max_lookback_slices. Each sub-list has n_top_syms tuples of (symbol, frequency) pairs, and is sorted in descending order of frequency. The frequency is the number of times the symbol appears in the top n_top_syms performance rankings of CAGR/UI, CAGR/retnStd and retnStd/UI.
# #### Therefore, symbols in the sub-list are the best performing symbols for the periods in days_lookbacks. Each sub-list corresponds to a tuple in max_lookback_slices. There are as many sub-lists as there are tuples in max_lookback_slices.
grp_top_set_syms_n_freq, grp_top_set_syms, dates_end_df_train = get_grp_top_syms_n_freq(
    df, sets_lookback_slices, days_lookbacks, n_top_syms, syms_start, syms_end, verbose
)

# #### print the best performing symbols for each set in sets_lookback_slices
for i, top_set_syms_n_freq in enumerate(grp_top_set_syms_n_freq):
    l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
    if verbose:
        print(f"max_lookback_slices:             {max_lookback_slices}")
        # print(f'set_lookback_slices: {sets_lookback_slices[i]}\n')
        print(
            f"set_lookback_slices {i + 1} of {len(sets_lookback_slices):>3}:    {sets_lookback_slices[i]}\n"
        )
        print(f"max_days_lookbacks:              {max_days_lookbacks}")
        print(f"df end date for days_lookbacks:  {dates_end_df_train[i]}")
        print(f"days_lookbacks:                  {days_lookbacks}")
        print(f"sym_freq_15:                     {l_sym_freq_cnt[0]}")
        print(f"sym_freq_14:                     {l_sym_freq_cnt[1]}")
        print(f"sym_freq_13:                     {l_sym_freq_cnt[2]}")
        print(f"sym_freq_12:                     {l_sym_freq_cnt[3]}")
        print(f"sym_freq_11:                     {l_sym_freq_cnt[4]}")
        print(f"sym_freq_10:                     {l_sym_freq_cnt[5]}")
        print(f"sym_freq_9:                      {l_sym_freq_cnt[6]}")
        print(f"sym_freq_8:                      {l_sym_freq_cnt[7]}")
        print(f"sym_freq_7:                      {l_sym_freq_cnt[8]}")
        print(f"sym_freq_6:                      {l_sym_freq_cnt[9]}")
        print(f"sym_freq_5:                      {l_sym_freq_cnt[10]}")
        print(f"sym_freq_4:                      {l_sym_freq_cnt[11]}")
        print(f"sym_freq_3:                      {l_sym_freq_cnt[12]}")
        print(f"sym_freq_2:                      {l_sym_freq_cnt[13]}\n")


# #### Evaluate performance of symbols in set_lookback_slices versus SPY
l_row_add_total = eval_grp_top_syms_n_freq(
    df, max_lookback_slices, sets_lookback_slices, grp_top_set_syms_n_freq, days_lookbacks, days_eval, n_samples, n_top_syms, syms_start, syms_end, verbose
)

for row_add_total in l_row_add_total:
    print(f"row_add_total: {row_add_total}")

if store_results:  # record results to df
    df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
    print(f"df_eval_results BEFORW store results:\n{df_eval_results}\n")

    for row_add_total in l_row_add_total:
        # print(f'row_add_total: {row_add_total}')
        df_eval_results.loc[len(df_eval_results)] = row_add_total
        print(f"appended row_add_total to df_eval_results:\n{row_add_total}\n")

    pickle_dump(df_eval_results, path_data_dump, fp_df_eval_results)
    print(f"Save results to: {fp_df_eval_results}")
    df_eval_results = pickle_load(path_data_dump, fp_df_eval_results)
    print(f"df_eval_results AFTER store results:\n{df_eval_results}\n")

max_days_lookbacks: 120
number of max_lookback_slices is equal to n_samples = 2
max_lookback_slices:
[(16, 136, 140), (17, 137, 141)]

number of sets in sets_lookback_slices is equal to n_samples = 2
sets_lookback_slices:
[[(121, 136, 140), (106, 136, 140), (76, 136, 140), (16, 136, 140)], [(122, 137, 141), (107, 137, 141), (77, 137, 141), (17, 137, 141)]]

days_lookbacks: [15, 30, 60, 120]
number of tuples in each "set of lookback slices" is equal to len(days_lookbacks): 4

########## 1 of 2 lb_slices: [(121, 136, 140), (106, 136, 140), (76, 136, 140), (16, 136, 140)] in sets_lookcak_slices ##########
days lookback:       15,  1 of 4 days_lookbacks: [15, 30, 60, 120]
lb_slices:           [(121, 136, 140), (106, 136, 140), (76, 136, 140), (16, 136, 140)]
lb_slice:            (121, 136, 140)
days eval:           4
iloc_start_train:    121
iloc_end_train:      136
date_start_df_train: 2023-11-24
date_end_df_train:   2023-12-14


perf_ranks: {'period-15': {'r_CAGR/UI': array(['HA', 'MBI', 'BIG', 'HPP', 'OSTK', 'SHV', 'FFWM', 'SMTC', 'LC',
       'WOR', 'TVTX', 'TNDM', 'RUN', 'NCLH', 'KRC', 'SPG', 'MARA', 'IVZ',
       'AX', 'ZG'], dtype=object), 'r_CAGR/retnStd': array(['HA', 'BIG', 'MBI', 'HPP', 'OSTK', 'RUN', 'MARA', 'FFWM', 'LC',
       'NCLH', 'TNDM', 'KRC', 'TVTX', 'VNO', 'SLG', 'ZG', 'WOR', 'BBWI',
       'SMTC', 'XENE'], dtype=object), 'r_retnStd/UI': array(['SHV', 'SPG', 'MBI', 'HD', 'IVZ', 'STLA', 'PSA', 'SWKS', 'SMTC',
       'KIM', 'CPT', 'PHM', 'MAS', 'BA', 'EXR', 'QRVO', 'STC', 'CG',
       'NTRS', 'SQ'], dtype=object)}}
most_freq_syms: [('MBI', 3), ('SMTC', 3), ('HA', 2), ('BIG', 2), ('HPP', 2), ('OSTK', 2), ('SHV', 2), ('FFWM', 2), ('LC', 2), ('WOR', 2), ('TVTX', 2), ('TNDM', 2), ('RUN', 2), ('NCLH', 2), ('KRC', 2), ('SPG', 2), ('MARA', 2), ('IVZ', 2), ('ZG', 2), ('AX', 1), ('VNO', 1), ('SLG', 1), ('BBWI', 1), ('XENE', 1), ('HD', 1), ('STLA', 1), ('PSA', 1), ('SWKS', 1), ('KIM', 1), ('CPT', 1), (

perf_ranks: {'period-30': {'r_CAGR/UI': array(['HA', 'AXGN', 'SQ', 'MBI', 'EFX', 'CG', 'PRAA', 'HPP', 'GDDY',
       'IBP', 'TNDM', 'EXPE', 'SHV', 'CCL', 'CYRX', 'MARA', 'SMTC',
       'TWLO', 'AMKR', 'KKR'], dtype=object), 'r_CAGR/retnStd': array(['HA', 'AXGN', 'SQ', 'HPP', 'TNDM', 'PRAA', 'MARA', 'CYRX', 'MBI',
       'CCL', 'FFWM', 'RUN', 'SMTC', 'IBP', 'NCLH', 'AMKR', 'CG', 'MTZ',
       'ZG', 'LUMN'], dtype=object), 'r_retnStd/UI': array(['SHV', 'GDDY', 'EFX', 'HA', 'MBI', 'CG', 'SQ', 'IT', 'EXPE',
       'FTSM', 'APH', 'AXGN', 'OC', 'KKR', 'IBP', 'SHW', 'UVV', 'EXR',
       'EXP', 'BLK'], dtype=object)}}
most_freq_syms: [('HA', 3), ('AXGN', 3), ('SQ', 3), ('MBI', 3), ('CG', 3), ('IBP', 3), ('EFX', 2), ('PRAA', 2), ('HPP', 2), ('GDDY', 2), ('TNDM', 2), ('EXPE', 2), ('SHV', 2), ('CCL', 2), ('CYRX', 2), ('MARA', 2), ('SMTC', 2), ('AMKR', 2), ('KKR', 2), ('TWLO', 1), ('FFWM', 1), ('RUN', 1), ('NCLH', 1), ('MTZ', 1), ('ZG', 1), ('LUMN', 1), ('IT', 1), ('FTSM', 1), ('APH', 1), ('OC', 1

perf_ranks: {'period-120': {'r_CAGR/UI': array(['SHV', 'FTSM', 'LRN', 'ANF', 'GPS', 'EDU', 'CLS', 'MLKN', 'VRNS',
       'HRB', 'NTNX', 'HIBB', 'QLYS', 'CBOE', 'MMYT', 'X', 'WSM', 'VST',
       'HOLI', 'AKAM'], dtype=object), 'r_CAGR/retnStd': array(['SHV', 'FTSM', 'GPS', 'ANF', 'EDU', 'CLS', 'MLKN', 'VRNS', 'NTNX',
       'MMYT', 'AEO', 'HRB', 'UBS', 'LRN', 'HIBB', 'QLYS', 'VST', 'WRK',
       'FFWM', 'WAL'], dtype=object), 'r_retnStd/UI': array(['SHV', 'FTSM', 'LRN', 'X', 'CBOE', 'CPRI', 'ANF', 'TDS', 'AIZ',
       'GPS', 'HIBB', 'HRB', 'QLYS', 'GD', 'ANET', 'WSM', 'AKAM', 'SUN',
       'EDU', 'HOLI'], dtype=object)}}
most_freq_syms: [('SHV', 3), ('FTSM', 3), ('LRN', 3), ('ANF', 3), ('GPS', 3), ('EDU', 3), ('HRB', 3), ('HIBB', 3), ('QLYS', 3), ('CLS', 2), ('MLKN', 2), ('VRNS', 2), ('NTNX', 2), ('CBOE', 2), ('MMYT', 2), ('X', 2), ('WSM', 2), ('VST', 2), ('HOLI', 2), ('AKAM', 2), ('AEO', 1), ('UBS', 1), ('WRK', 1), ('FFWM', 1), ('WAL', 1), ('CPRI', 1), ('TDS', 1), ('AIZ', 1), ('GD', 1)

perf_ranks: {'period-30': {'r_CAGR/UI': array(['HA', 'MBI', 'MARA', 'AXGN', 'SQ', 'CG', 'TNDM', 'EFX', 'PRAA',
       'SHV', 'WOR', 'TWLO', 'EXR', 'IBP', 'BA', 'KKR', 'CCL', 'SMTC',
       'AMKR', 'RUN'], dtype=object), 'r_CAGR/retnStd': array(['HA', 'MARA', 'MBI', 'TNDM', 'AXGN', 'PRAA', 'SQ', 'RUN', 'WOR',
       'ELF', 'CCL', 'CYRX', 'CG', 'SLAB', 'AMKR', 'HPP', 'LUMN', 'BA',
       'TWLO', 'ENPH'], dtype=object), 'r_retnStd/UI': array(['MBI', 'SHV', 'EFX', 'HA', 'CG', 'FTSM', 'SQ', 'APH', 'OC', 'KKR',
       'SHW', 'EXR', 'AXGN', 'IBP', 'EXP', 'BLK', 'BURL', 'DIA', 'GPS',
       'TWLO'], dtype=object)}}
most_freq_syms: [('HA', 3), ('MBI', 3), ('AXGN', 3), ('SQ', 3), ('CG', 3), ('TWLO', 3), ('MARA', 2), ('TNDM', 2), ('EFX', 2), ('PRAA', 2), ('SHV', 2), ('WOR', 2), ('EXR', 2), ('IBP', 2), ('BA', 2), ('KKR', 2), ('CCL', 2), ('AMKR', 2), ('RUN', 2), ('SMTC', 1), ('ELF', 1), ('CYRX', 1), ('SLAB', 1), ('HPP', 1), ('LUMN', 1), ('ENPH', 1), ('FTSM', 1), ('APH', 1), ('OC', 1), ('SHW', 1), (

perf_ranks: {'period-120': {'r_CAGR/UI': array(['SHV', 'FTSM', 'ANF', 'LRN', 'GPS', 'CLS', 'QLYS', 'MLKN', 'EDU',
       'HIBB', 'NTNX', 'CBOE', 'VRNS', 'HRB', 'X', 'MMYT', 'WSM', 'VST',
       'AKAM', 'HOLI'], dtype=object), 'r_CAGR/retnStd': array(['SHV', 'FTSM', 'GPS', 'ANF', 'CLS', 'MLKN', 'VRNS', 'NTNX', 'MMYT',
       'EDU', 'QLYS', 'AEO', 'HIBB', 'VST', 'LRN', 'WRK', 'UBS', 'HRB',
       'FNF', 'FFWM'], dtype=object), 'r_retnStd/UI': array(['SHV', 'FTSM', 'LRN', 'X', 'CPRI', 'CBOE', 'ANF', 'TDS', 'AIZ',
       'GPS', 'HRB', 'HIBB', 'QLYS', 'GD', 'ANET', 'WSM', 'SUN', 'AKAM',
       'EDU', 'HOLI'], dtype=object)}}
most_freq_syms: [('SHV', 3), ('FTSM', 3), ('ANF', 3), ('LRN', 3), ('GPS', 3), ('QLYS', 3), ('EDU', 3), ('HIBB', 3), ('HRB', 3), ('CLS', 2), ('MLKN', 2), ('NTNX', 2), ('CBOE', 2), ('VRNS', 2), ('X', 2), ('MMYT', 2), ('WSM', 2), ('VST', 2), ('AKAM', 2), ('HOLI', 2), ('AEO', 1), ('WRK', 1), ('UBS', 1), ('FNF', 1), ('FFWM', 1), ('CPRI', 1), ('TDS', 1), ('AIZ', 1), ('GD', 1)